In [ ]:
import os
COLAB = True
work_dir = "/home/changl25/ziplora-pytorch"
os.chdir(work_dir)
!pwd
import sys
from pathlib import Path

# Playground

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np 
import torch
from diffusers import AutoencoderKL, StableDiffusionXLPipeline
from transformers import AutoTokenizer
MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"
device = "cuda" if torch.cuda.is_available() else "cpu"
weight_dtype = torch.float16
rank=64
%load_ext autoreload
%autoreload 2

from inverse_ziplora_comb_separate import utils
from inverse_ziplora_comb_separate.pipeline_stable_diffusion_xl_seperate import StableDiffusionXLSeperatePipeline


In [ ]:
from train_inverse_ziplora_layer_column import import_model_class_from_model_name_or_path

tokenizer_one = AutoTokenizer.from_pretrained(
    MODEL_ID,
    subfolder="tokenizer",
    use_fast=False,
)
tokenizer_two = AutoTokenizer.from_pretrained(
    MODEL_ID,
    subfolder="tokenizer_2",
    use_fast=False,
)
text_encoder_cls_one = import_model_class_from_model_name_or_path(
    MODEL_ID, revision=None
)
text_encoder_cls_two = import_model_class_from_model_name_or_path(
    MODEL_ID, subfolder="text_encoder_2", revision=None
)
text_encoder_one = text_encoder_cls_one.from_pretrained(
    MODEL_ID, subfolder="text_encoder",
)
text_encoder_two = text_encoder_cls_two.from_pretrained(
    MODEL_ID, subfolder="text_encoder_2",
)
text_encoder_one.requires_grad_(False)
text_encoder_one.to(device, dtype=torch.float16)

text_encoder_two.requires_grad_(False)
text_encoder_two.to(device, dtype=torch.float16)
tokenizers = [tokenizer_one, tokenizer_two]
text_encoders = [text_encoder_one, text_encoder_two]

vae = AutoencoderKL.from_pretrained(
            MODEL_ID,
            subfolder="vae",
            revision=None
    )

In [ ]:
def log_validation(pipeline, prompt, prompt_content, prompt_style, seed=0):
    generator = torch.Generator(device=device).manual_seed(seed)
    # Currently the context determination is a bit hand-wavy. We can improve it in the future if there's a better
    # way to condition it. Reference: https://github.com/huggingface/diffusers/pull/7126#issuecomment-1968523051
    if pipeline.__class__.__name__ == 'StableDiffusionXLSeperatePipeline':
        pipeline_args = {"prompt": prompt, 
                        "prompt_content": prompt_content, 
                        "prompt_style": prompt_style}
    else: 
        pipeline_args = {"prompt": prompt}
    print(pipeline_args)    
    images = [pipeline(**pipeline_args, generator=generator, num_inference_steps=50).images[0] for _ in range(4)]
    return images

In [ ]:
def show_img(images):
    np_images = np.hstack([np.asarray(img) for img in images])
    plt.figure(figsize=(10,10))
    plt.imshow(np_images)
    plt.axis("off")
    plt.show()

In [ ]:
def show_single_img(image):
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

In [ ]:
def load_pipeline_unet(output_dir, output_dir2):
    content_unet_path = f"{output_dir}_content"
    style_unet_path = f"{output_dir2}_style"
    weight_content_path=f"{output_dir}_merger_content.pth"
    weight_style_path=f"{output_dir2}_merger_style.pth"
    pipeline = utils.load_pipeline_from_sdxl(
        MODEL_ID, vae = vae
    )
    pipeline.unet = utils.insert_inverse_ziplora_to_unet(pipeline.unet,
        content_unet_path, 
        style_unet_path, 
        weight_content_path,
        weight_style_path,      
        rank=64)
    pipeline = pipeline.to(device, dtype=weight_dtype)
    
    return pipeline

In [ ]:
def remove_pipeline(pipeline):
    del pipeline
    torch.cuda.empty_cache()

## Content / Style Individual Generation

In [ ]:
content = "rose"
style = "pop art"

### Use Pipeline

In [ ]:
model_dir = ""
pipeline = StableDiffusionXLPipeline.from_pretrained(
    MODEL_ID,
    vae=vae,
    torch_dtype=weight_dtype,
)
pipeline.load_lora_weights(f"{model_dir}_content")
prompt_content_main = f"A photo of monadikos {content}"
prompt_content = f"A photo of monadikos {content}"
imgs_content = log_validation(pipeline, prompt_content, prompt_content, prompt_content)
show_img(imgs_content)

In [ ]:
pipeline.load_lora_weights(f"{model_dir}_style")
prompt_style = f"a dog in {style} style"
pipeline.unet = utils.inverse_ziplora_set_forward_type(pipeline.unet, type="style")

imgs_style = log_validation(pipeline, prompt_style, prompt_style, prompt_style)
show_img(imgs_style)

### Use UnZipLoRA Pipeline

In [ ]:
model_dir_content = ""
model_dir_style = ""
pipeline = load_pipeline_unet(model_dir_content, model_dir_style)


In [ ]:
prompt_content_main = f"A photo of monadikos {content}"
prompt_content = f"A photo of monadikos {content}"
pipeline.unet = utils.inverse_ziplora_set_forward_type(pipeline.unet, type="content")

# imgs_content = log_validation(pipeline, prompt_content_main, prompt_content, prompt_content)
imgs_content = log_validation(pipeline, prompt_content, prompt_content, prompt_content)
show_img(imgs_content)

In [ ]:
prompt_style = f"a dog in {style} style"
pipeline.unet = utils.inverse_ziplora_set_forward_type(pipeline.unet, type="style")

imgs_style = log_validation(pipeline, prompt_style, prompt_style, prompt_style)
show_img(imgs_style)

## Combine Generation

In [ ]:
prompt_combine = f"a monadikos {content} in {style} style on a skateboard"
prompt_combine_content = f"a monadikos {content} on a skateboard"
prompt_combine_style = f"a {content} in {style} style on a skateboard"

pipeline.unet = utils.inverse_ziplora_set_forward_type(pipeline.unet, type="both")
imgs = log_validation(pipeline, prompt_combine, prompt_combine_content, prompt_combine_style)
show_img(imgs)

## Cross-image Generation

In [ ]:
model_dir_content = ""
model_dir_style = ""
content_prompt = "rose"
style_prompt = "geometry"

In [ ]:
pipeline = load_pipeline_unet(model_dir_content, model_dir_style)

In [ ]:
prompt_combine = f"a monadikos {content_prompt} in {style_prompt} style"
prompt_combine_content = f"a monadikos {content_prompt} "
prompt_combine_style = f"a {content_prompt} in {style_prompt} style"

pipeline.unet = utils.inverse_ziplora_set_forward_type(pipeline.unet, type="both")
imgs = log_validation(pipeline, prompt_combine, prompt_combine_content, prompt_combine_style)
show_img(imgs)